# Md Abrar Jahin

# Modelling

In [1]:
#@title
import sklearn
from sklearn.model_selection import GridSearchCV
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances, silhouette_samples, silhouette_score, confusion_matrix, roc_curve, auc
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering 

from statsmodels.stats.outliers_influence import variance_inflation_factor
from time import time
from datetime import datetime
import matplotlib.cm as cm
from scipy.spatial.distance import cdist
from yellowbrick.cluster import KElbowVisualizer
from datetime import datetime
from joblib import Parallel, delayed

import seaborn as sns
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import joblib

#Import libraries:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from dask.distributed import Client

# It's recommended to use dask_cuda for GPU assignment
#!pip install dask dask_cuda dask-ml 
#!pip install tensorflow
#import os
#os.kill(os.getpid(), 9)
#from dask_cuda import LocalCUDACluster
from dask import array as da

%matplotlib inline

In [2]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title
import pandas as pd
import numpy as np
import gensim 
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_data.csv')

In [4]:
#@title
#after manual check
negative_num=len(df[df['senti_vader'] < 0])
print("negative:", negative_num)
neutral_num=len(df[df['senti_vader'] == 0])
print("neutral", neutral_num)
positive_num=len(df[df['senti_vader'] > 0])
print("positive", positive_num) 

negative: 1000
neutral 1000
positive 1000


In [5]:
df['lemma_sentence(with POS)']

0         kid not wana homework sparkle covid sparkle ...
1         not mess covid wear mask time london united ...
2         problem airfield cost remain open verse pote...
3         remain astonished stock market not face riot...
4         dear lord jesus christ grace listen prayer l...
                              ...                        
2995      lockdown not deal covid deal stupidity enrag...
2996      regulator refuse approve mass covid test sch...
2997      covid vaccine week ago manchester real perso...
2998      coronavirustrain covid covid wearamask weara...
2999       news cycle number covid death car crash brexit
Name: lemma_sentence(with POS), Length: 3000, dtype: object

In [6]:
#@title
#BoW
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from nltk.tokenize import word_tokenize

def bag_of_words(df):
#     bow_vectorizer = CountVectorizer(max_df=0.90, min_df=0.2, stop_words=None, tokenizer=word_tokenize) 
    bow_vectorizer = CountVectorizer()
    bow = bow_vectorizer.fit_transform(df['lemma_sentence(with POS)']) 
    #print(bow_vectorizer.get_feature_names())
    print(len(bow_vectorizer.vocabulary_))
    return bow

df_bow=bag_of_words(df)
df_bow.shape

7222


(3000, 7222)

In [7]:

#@title
#TF-IDF
def tf_idf(df):
#     tf_idf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.2, stop_words=None, tokenizer=word_tokenize, norm='l2') 
    tf_idf_vectorizer = TfidfVectorizer(norm='l2') #extract features
    tfidf = tf_idf_vectorizer.fit_transform(df['lemma_sentence(with POS)']) #vectors
    return tfidf
df_tfidf=tf_idf(df)
df_tfidf.shape

(3000, 7222)

In [8]:
#@title
#Word2vec
#reference：https://www.pythonf.cn/read/93491
#https://github.com/Shwetago/Sentiment_Analysis/blob/master/Twitter_Sentiment_Analysis.ipynb
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
Tokenize_tweet = df['lemma_sentence(with POS)'].apply(word_tokenize)
print(Tokenize_tweet)

Model_W2V = gensim.models.Word2Vec(Tokenize_tweet, vector_size=200, #features
                                   window=5, 
                                   min_count=1, 
                                   sg=1,  #skip-gram model
                                   hs=0,
                                   negative=10, 
                                   workers=2, 
                                   seed=34) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0       [kid, not, wana, homework, sparkle, covid, spa...
1       [not, mess, covid, wear, mask, time, london, u...
2       [problem, airfield, cost, remain, open, verse,...
3       [remain, astonished, stock, market, not, face,...
4       [dear, lord, jesus, christ, grace, listen, pra...
                              ...                        
2995    [lockdown, not, deal, covid, deal, stupidity, ...
2996    [regulator, refuse, approve, mass, covid, test...
2997    [covid, vaccine, week, ago, manchester, real, ...
2998    [coronavirustrain, covid, covid, wearamask, we...
2999    [news, cycle, number, covid, death, car, crash...
Name: lemma_sentence(with POS), Length: 3000, dtype: object


In [9]:
#@title
#Each word can get its own vector. The representation of a tweets can the vector sum of each word divided by the total number(average) 
#or just the sum of each word vector
def word2vec_tweet(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size))
        vector_cnt += 1
    return vector/vector_cnt  #average for tweets

def word2vec_tweet_2(tokens, size):
    vector=np.zeros(size).reshape((1,size))
    vector_cnt = 0
    for word in tokens:
        vector += Model_W2V.wv[word].reshape((1, size))
    return vector  #sum of tweets

tweet_arr=np.zeros((len(Tokenize_tweet), 200))

for i in range (len(Tokenize_tweet)):
    tweet_arr[i,:] = word2vec_tweet(Tokenize_tweet[i], 200)
tweet_vec_df = pd.DataFrame(tweet_arr)

In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
import tensorflow_hub as hub
#!pip install transformers
#from transformers import BertTokenizer, TFBertModel, AutoTokenizer, TFAutoModel
import tensorflow as tf
#!pip install sentence_transformers
#from sentence_transformers import SentenceTransformer

# HashingVectorizer
hash_vec = HashingVectorizer(n_features=1000)
df_hash = hash_vec.fit_transform(df['lemma_sentence(with POS)'])
print("HashingVectorizer output shape:", df_hash.shape)

'''
# Load pre-trained SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Generate document-level embeddings using pre-trained SentenceTransformer
df_sbert = sbert_model.encode(df['lemma_sentence(with POS)'].tolist(), batch_size=32, show_progress_bar=True)
print("SentenceTransformer output shape:", df_sbert.shape)
'''

HashingVectorizer output shape: (3000, 1000)


'\n# Load pre-trained SentenceTransformer\nsbert_model = SentenceTransformer(\'bert-base-nli-mean-tokens\')\n\n# Generate document-level embeddings using pre-trained SentenceTransformer\ndf_sbert = sbert_model.encode(df[\'lemma_sentence(with POS)\'].tolist(), batch_size=32, show_progress_bar=True)\nprint("SentenceTransformer output shape:", df_sbert.shape)\n'

In [11]:
#parameters in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] #tree number
max_features = ['auto', 'sqrt','log2']
max_depth = [10,20,30,40,50]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5, 10]

# Create the param grid
param_grid_forest = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid_forest)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 5, 10]}


In [12]:

#parameters in MNB
param_grid_nb = {'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0],
                'fit_prior':[True, False]}
print(param_grid_nb)

{'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0], 'fit_prior': [True, False]}


In [13]:
#parameters in SVC
# c_list=list(range(1,51))
param_grid_svc = {'C': [1, 10, 100, 1000],
                  'kernel': ['linear','poly','rbf','sigmoid'],
                  'degree': [1,2,3,4]}
print(param_grid_svc)

{'C': [1, 10, 100, 1000], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4]}


In [14]:
#parameters in Gradient Boosting Machine(GBM)
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_features = ['auto', 'sqrt','log2']
max_depth = [10,20,30,40]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 5, 10]

#Create the param grid for GBM
param_grid_gbm = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf}
print(param_grid_gbm)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 5, 10]}


In [15]:
#Parameters in XGBoost
learning_rate = [0.01, 0.1, 0.5, 1, 0.3]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [20,30,40,50]
min_child_weight = [1, 3, 5, 7]
gamma = [0.0, 0.1, 0.2, 0.3]
subsample = [0.6, 0.7, 0.8, 0.9]
colsample_bytree = [0.6, 0.7, 0.8, 0.9]
booster=['gbtree','gblinear']

#Create the param grid for XGBoost
param_grid_xgb = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_depth': max_depth,
'min_child_weight': min_child_weight,
'objective': ['multi:softmax','multi:softprob'],
'gamma': gamma,
'subsample': subsample,
'colsample_bytree': colsample_bytree,
'booster': booster}

print(param_grid_xgb)

{'learning_rate': [0.01, 0.1, 0.5, 1, 0.3], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_depth': [20, 30, 40, 50], 'min_child_weight': [1, 3, 5, 7], 'objective': ['multi:softmax', 'multi:softprob'], 'gamma': [0.0, 0.1, 0.2, 0.3], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'booster': ['gbtree', 'gblinear']}


In [16]:
#Parameters in LightGBM
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [5, 10, 15, 20]
num_leaves = [10, 20, 30, 40]
min_child_samples = [1, 3, 5, 7]
subsample = [0.6, 0.7, 0.8, 0.9]
colsample_bytree = [0.6, 0.7, 0.8, 0.9]

#Create the param grid for LightGBM
param_grid_lgbm = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'max_depth': max_depth,
'num_leaves': num_leaves,
'objective': ['multiclass'],
'min_child_samples': min_child_samples,
'subsample': subsample,
'colsample_bytree': colsample_bytree}
print(param_grid_lgbm)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'max_depth': [5, 10, 15, 20], 'num_leaves': [10, 20, 30, 40], 'objective': ['multiclass'], 'min_child_samples': [1, 3, 5, 7], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}


In [17]:
#parameters in CatBoost
learning_rate = [0.01, 0.1, 0.5, 1]
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)] # tree number
max_depth = [10,20,30,40]
l2_leaf_reg = [1, 3, 5, 7, 9]

#Create the param grid for CatBoost
param_grid_catboost = {'learning_rate': learning_rate,
'n_estimators': n_estimators,
'loss_function': ['MultiClass'],
'max_depth': max_depth,
'l2_leaf_reg': l2_leaf_reg}
print(param_grid_catboost)

{'learning_rate': [0.01, 0.1, 0.5, 1], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'loss_function': ['MultiClass'], 'max_depth': [10, 20, 30, 40], 'l2_leaf_reg': [1, 3, 5, 7, 9]}


In [18]:
#@title
from sklearn.metrics import classification_report 
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgbm
!pip install catboost
from catboost import CatBoostClassifier

model_forest = RandomForestClassifier()
model_nb = MultinomialNB()
model_svc = SVC()
model_xgb = xgb.XGBClassifier()
model_gbm = GradientBoostingClassifier()
model_lgbm = lgbm.LGBMClassifier()
model_cat = CatBoostClassifier()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.4 MB/s eta 0:00:00


# TextBlob

In [37]:
#@title
#BoW for three classification models
#split the train and test datasets
from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2)
#x_train, x_test, y_train, y_test = train_test_split(df_tfidf, df['senti_textblob'],test_size = 0.2)
x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2)

In [38]:
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

# create an instance of the OneHotEncoder class
encoder = OneHotEncoder(sparse=False)

# fit the encoder to the labels and transform the labels into one-hot encoded vectors
#y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
#y_test_encoded = encoder.fit_transform(y_test.values.reshape(-1, 1))
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [40]:
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

from sklearn.preprocessing import MinMaxScaler

# Scale the input data
#scaler = MinMaxScaler()
#x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
        if(name!='xgb' and name!='nb'):
            RandomGrid.fit(x_train, y_train)
        elif(name=='nb'):
            RandomGrid.fit(x_train, y_train)
        else:
            RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
        print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
        print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))

Best params for rf:
{'n_estimators': 10, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 40}
Classification report for rf:
              precision    recall  f1-score   support

        -1.0       0.91      0.31      0.47       156
         0.0       0.59      0.97      0.73       215
         1.0       0.76      0.63      0.69       229

    accuracy                           0.67       600
   macro avg       0.75      0.64      0.63       600
weighted avg       0.74      0.67      0.65       600

Best params for nb:
{'fit_prior': True, 'alpha': 0.5}
Classification report for nb:
              precision    recall  f1-score   support

        -1.0       0.57      0.52      0.54       156
         0.0       0.72      0.70      0.71       215
         1.0       0.65      0.71      0.68       229

    accuracy                           0.66       600
   macro avg       0.65      0.64      0.64       600
weighted avg       0.65      0.66      0.65      

# Vader

In [19]:
#@title
#BoW for three classification models
#split the train and test datasets
from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2)
#x_train, x_test, y_train, y_test = train_test_split(df_tfidf, df['senti_textblob'],test_size = 0.2)
x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_vader'],test_size = 0.2)

In [20]:
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

# create an instance of the OneHotEncoder class
encoder = OneHotEncoder(sparse=False)

# fit the encoder to the labels and transform the labels into one-hot encoded vectors
#y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
#y_test_encoded = encoder.fit_transform(y_test.values.reshape(-1, 1))
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [21]:
#'''
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

#from sklearn.preprocessing import MinMaxScaler

# Scale the input data
#scaler = MinMaxScaler()
#x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
      if(name!='xgb' and name!='nb'):
        RandomGrid.fit(x_train, y_train)
      elif(name=='nb'):
        RandomGrid.fit(x_train, y_train)
      else:
        RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
      print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
      print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
      print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))

#'''

Best params for rf:
{'n_estimators': 190, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 40}
Classification report for rf:
              precision    recall  f1-score   support

        -1.0       0.74      0.54      0.62       202
         0.0       0.58      0.91      0.71       188
         1.0       0.78      0.60      0.68       210

    accuracy                           0.68       600
   macro avg       0.70      0.68      0.67       600
weighted avg       0.71      0.68      0.67       600

Best params for nb:
{'fit_prior': True, 'alpha': 0.5}
Classification report for nb:
              precision    recall  f1-score   support

        -1.0       0.65      0.65      0.65       202
         0.0       0.64      0.62      0.63       188
         1.0       0.66      0.67      0.66       210

    accuracy                           0.65       600
   macro avg       0.65      0.65      0.65       600
weighted avg       0.65      0.65      0.65      

# SentiWordNet

In [19]:
#@title
#BoW for three classification models
#split the train and test datasets
from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_textblob'],test_size = 0.2)
#x_train, x_test, y_train, y_test = train_test_split(df_tfidf, df['senti_textblob'],test_size = 0.2)
x_train, x_test, y_train, y_test = train_test_split(df_bow, df['senti_wordnet'],test_size = 0.2)

In [20]:
#@title
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

# create an instance of the OneHotEncoder class
encoder = OneHotEncoder(sparse=False)

# fit the encoder to the labels and transform the labels into one-hot encoded vectors
#y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))
#y_test_encoded = encoder.fit_transform(y_test.values.reshape(-1, 1))
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [21]:
#'''
### ALL MODELS #######
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

#from sklearn.preprocessing import MinMaxScaler

# Scale the input data
#scaler = MinMaxScaler()
#x_train_scaled = scaler.fit_transform(x_train)

models_list = [model_forest,model_nb,model_svc,model_gbm,model_lgbm,model_xgb,model_cat]
models_name = ['rf','nb','svc','gbm','lgbm','xgb','cat']
param_grids = [param_grid_forest,param_grid_nb,param_grid_svc,param_grid_gbm,param_grid_lgbm,param_grid_xgb,param_grid_catboost]

for model,name,param_grid in zip(models_list, models_name, param_grids):
    #best parameters for model (with BoW)
    RandomGrid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=2, verbose=0, n_jobs=4, random_state=40)
    with parallel_backend('multiprocessing'):
      if(name!='xgb' and name!='nb'):
        RandomGrid.fit(x_train, y_train)
      elif(name=='nb'):
        RandomGrid.fit(x_train, y_train)
      else:
        RandomGrid.fit(x_train, y_train_encoded)
    print('Best params for ' + name+':')
    print(RandomGrid.best_params_)

    print('Classification report for ' + name+':')
    if(name!='xgb' and name!='nb'):
      print(classification_report(y_test, RandomGrid.predict(x_test)))
    elif(name=='nb'):
      print(classification_report(y_test, RandomGrid.predict(x_test)))
    else:
      print(classification_report(y_test_encoded, RandomGrid.predict(x_test)))

#'''

Best params for rf:
{'n_estimators': 10, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 40}
Classification report for rf:
              precision    recall  f1-score   support

        -1.0       0.50      0.90      0.65       246
         0.0       0.00      0.00      0.00       116
         1.0       0.73      0.50      0.59       238

    accuracy                           0.57       600
   macro avg       0.41      0.47      0.41       600
weighted avg       0.50      0.57      0.50       600

Best params for nb:
{'fit_prior': True, 'alpha': 0.5}
Classification report for nb:
              precision    recall  f1-score   support

        -1.0       0.62      0.74      0.67       246
         0.0       0.32      0.18      0.23       116
         1.0       0.67      0.68      0.68       238

    accuracy                           0.61       600
   macro avg       0.54      0.53      0.53       600
weighted avg       0.58      0.61      0.59      